## 1. Environment Dependencies

In [1]:
!pip install tensorflow==2.5.0 gym keras-rl2 gym[atari]

     -------------------------------------- 422.6/422.6 MB 1.7 MB/s eta 0:00:00
     -------------------------------------- 132.0/132.0 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 2.9/2.9 MB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 13.3/13.3 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 42.6/42.6 kB 2.0 MB/s eta 0:00:00
     -------------------------------------- 462.4/462.4 kB 1.9 MB/s eta 0:00:00
     ---------------------------------------- 1.2/1.2 MB 2.4 MB/s eta 0:00:00
     ---------------------------------------- 2.7/2.7 MB 3.1 MB/s eta 0:00:00
     -------------------------------------- 950.7/950.7 kB 1.4 MB/s eta 0:00:00
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4832

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'g:\\program\\lib\\site-packages\\numpy\\core\\_multiarray_tests.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



## 2. Random Environment with OpenAI Gym

In [2]:
import gym
import random

AttributeError: module 'numpy' has no attribute 'random'

In [ ]:
env = gym.make('SpaceInvaders-v0')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [ ]:
env.unwrapped.get_action_meanings()

In [ ]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice([0,1,2,3,4,5])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

## 3. Creating Keras Deep Learning Model 

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [ ]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [ ]:
del model

In [ ]:
model = build_model(height, width, channels, actions)

In [ ]:
model.summary()

## 4. Building KerasRL Agent 

In [ ]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [ ]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4))

In [ ]:
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

In [ ]:
scores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward']))

## 5. Saving and Reloading Agent 

In [ ]:
dqn.save_weights('SavedWeights/10k-Fast/dqn_weights.h5f')

In [ ]:
del model, dqn

In [ ]:
dqn.load_weights('SavedWeights/1m/dqn_weights.h5f')